In [1]:
import torch
from unsloth import FastLanguageModel
import weave; weave.init('think_test')

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/lewis/github/unsloth/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


weave: Logged in as Weights & Biases user: lewis-won.
weave: View Weave data at https://wandb.ai/lewis-won-sear/think_test/weave
INFO:weave.trace.init_message:Logged in as Weights & Biases user: lewis-won.
View Weave data at https://wandb.ai/lewis-won-sear/think_test/weave


In [2]:
# --- Model Setup Variables ---
BASE_MODEL_NAME = "unsloth/Qwen3-0.6B"
max_seq_length = 2048
dtype = None
load_in_4bit = False


In [3]:
# --- Load Model and Tokenizer ---
BASE_MODEL, TOKENIZER = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
BASE_MODEL.eval().to("cuda")
FastLanguageModel.for_inference(BASE_MODEL)


==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024, padding_idx=151654)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3

In [4]:
# --- Prompt Preparation Functions ---
def make_prompt(instruction):
    return [{"role": "user", "content": instruction}]


def apply_chat_template(prompt, tokenizer, enable_thinking=True):
    messages = make_prompt(prompt)
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=enable_thinking,
    )

In [5]:
@weave.op
def generate_response(prompt, enable_thinking=True):
    prompt_text = apply_chat_template(prompt, TOKENIZER, enable_thinking)
    inputs = TOKENIZER([prompt_text], return_tensors="pt").to("cuda")
    with torch.no_grad():
        gen_output = BASE_MODEL.generate(
            **inputs,
            max_new_tokens=128,
            use_cache=False,
            temperature=0.7,
            top_p=0.8,
            top_k=20,
            min_p=0.0,
        )
    output_text = TOKENIZER.decode(gen_output[0], skip_special_tokens=True)
    return output_text


In [6]:
# --- Test Prompts ---
math_question = "What is 256 multiplied by 17?"
math_question_no_think = "/no_think\nWhat is 256 multiplied by 17?"


print("=== enable_thinking=True (default) ===")
output1 = generate_response(math_question, enable_thinking=True)
print(output1.strip())
print()


print("=== enable_thinking=False ===")
output2 = generate_response(math_question, enable_thinking=False)
print(output2.strip())
print()


print("=== enable_thinking=True + /no_think in prompt ===")
output3 = generate_response(math_question_no_think, enable_thinking=True)
print(output3.strip())


=== enable_thinking=True (default) ===


weave: 🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-08fc-7fe1-8608-b6379f3e7e42
INFO:weave.trace.weave_client:🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-08fc-7fe1-8608-b6379f3e7e42
weave: 🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-14fb-776d-92c0-eacaa60a1812
INFO:weave.trace.weave_client:🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-14fb-776d-92c0-eacaa60a1812


user
What is 256 multiplied by 17?
assistant
<think>
Okay, so I need to figure out what 256 multiplied by 17 is. Hmm, let's see. I remember that multiplying numbers can be done by breaking them down into smaller parts. Maybe I can use the standard multiplication algorithm here. Let me try to remember how that works.

First, let me write down the numbers to visualize them better. 256 multiplied by 17. I think 17 can be broken down into 10 and 7, right? Because 10 plus 7 equals 17. So maybe that helps me do the multiplication step by step

=== enable_thinking=False ===


weave: 🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-1b56-7281-88d8-198a88dd6eca
INFO:weave.trace.weave_client:🍩 https://wandb.ai/lewis-won-sear/think_test/r/call/0198ff39-1b56-7281-88d8-198a88dd6eca


user
What is 256 multiplied by 17?
assistant
<think>

</think>

To calculate **256 × 17**, we can break it down:

$$
256 \times 17 = 256 \times (10 + 7) = 256 \times 10 + 256 \times 7
$$

$$
= 2560 + 1792 = 4352
$$

So, **256 × 17 = 4352**.

=== enable_thinking=True + /no_think in prompt ===
user
/no_think
What is 256 multiplied by 17?
assistant
<think>

</think>

To find the product of 256 and 17:

$$
256 \times 17 = 256 \times (10 + 7) = 256 \times 10 + 256 \times 7 = 2560 + 1792 = 4352
$$

So, **256 multiplied by 17 is 4352**.


In [7]:
import random
import numpy as np
import torch


SEED = 3407


random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [8]:
# ====== REST OF SCRIPT ======
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


max_seq_length = 2048
dtype = None
load_in_4bit = False
MODEL_NAME = "unsloth/Qwen3-0.6B"
SAVE_DIR = "lora_model"


In [9]:
# Load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=SEED,
    use_rslora=False,
    loftq_config=None,
)


==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.8.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [10]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        if input_text.strip():
            user_message = f"{instruction}\n\n{input_text}"
        else:
            user_message = instruction
        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": output},
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
            enable_thinking=False
        )
        texts.append(text)
    return {"text": texts}


In [11]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
half_len = len(dataset) // 2
dataset = dataset.select(range(half_len))
dataset = dataset.map(formatting_prompts_func, batched=True, num_proc=2)


Map (num_proc=2): 100%|██████████| 25880/25880 [00:00<00:00, 33469.49 examples/s]


In [12]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=SEED,  # Make sure to set this!
        output_dir="outputs",
        report_to="wandb",
    ),
)
trainer.train()


Unsloth: Tokenizing ["text"] (num_proc=16): 100%|██████████| 25880/25880 [00:02<00:00, 11114.12 examples/s]
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 25,880 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)
wandb: Currently logged in as: lewis-won (lewis-won-sear) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Initializing weave.


/home/lewis/github/unsloth/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

weave: Logged in as Weights & Biases user: lewis-won.
weave: View Weave data at https://wandb.ai/lewis-won-sear/huggingface/weave
INFO:weave.trace.init_message:Logged in as Weights & Biases user: lewis-won.
View Weave data at https://wandb.ai/lewis-won-sear/huggingface/weave


Step,Training Loss,entropy
1,2.082800,0
2,2.313800,No Log
3,1.426100,No Log
4,1.680800,No Log
5,2.227000,No Log
6,1.779800,No Log
7,1.866300,No Log
8,1.835800,No Log
9,1.946300,No Log
10,1.999100,No Log


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=60, training_loss=1.6049276173114777, metrics={'train_runtime': 34.866, 'train_samples_per_second': 13.767, 'train_steps_per_second': 1.721, 'total_flos': 327517470720000.0, 'train_loss': 1.6049276173114777, 'epoch': 0.01854714064914992})

In [13]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [14]:
user_query = "Continue the Fibonacci sequence.\n\n1, 1, 2, 3, 5, 8"
messages = [
    {"role": "user", "content": user_query},
]
prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")


outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    use_cache=False,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    min_p=0.0,
)


print("\n=========== Output from in-memory model (just trained):")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



=========== Output from in-memory model (just trained):
user
Continue the Fibonacci sequence.

1, 1, 2, 3, 5, 8
assistant
<think>

</think>

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987


In [15]:
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)


('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [16]:
del model
del tokenizer
torch.cuda.empty_cache()

==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Input IDs of shape torch.Size([1, 2049]) with length 2049 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 2050]) with length 2050 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 2051]) with length 2051 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 2052]) with length 2052 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 2053]) with length 2053 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 20


=========== Output from reloaded model (after save/load):
user
Continue the Fibonacci sequence.

1, 1, 2, 3, 5, 8
assistant
<think>

</think>

The Fibonacci sequence is defined as follows:

1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 63245986, 102334155, 165580141, 267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073, 4807526976, 7778742049, 12586269025, 20365011074, 32951280109, 53316291183, 86267571292, 139583862475, 225851433767, 365435306242, 591286739909, 956722046151, 1548008786060, 2504730832211, 4052739618271, 6557470450482, 10610209068753, 17167689519235, 27777898587988, 44945588107223, 72723486695211, 117679074702434, 190402561397645, 308081636100079, 508484207497724, 816565843597803, 1325049050095527, 2141614893693320, 3466663943788847, 5608288837482167, 907495278127101

In [17]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=SAVE_DIR,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
FastLanguageModel.for_inference(model)


prompt2 = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
inputs2 = tokenizer([prompt2], return_tensors="pt").to("cuda")


outputs2 = model.generate(
    **inputs2,
    max_new_tokens=2048,
    use_cache=False,
    temperature=0.7,
    top_p=0.8,
    top_k=20,
    min_p=0.0,
)


print("\n=========== Output from reloaded model (after save/load):")
print(tokenizer.decode(outputs2[0], skip_special_tokens=True))

==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

=========== Output from reloaded model (after save/load):
user
Continue the Fibonacci sequence.

1, 1, 2, 3, 5, 8
assistant
<think>

</think>

The next Fibonacci numbers after 1, 1, 2, 3, 5, 8 would be 13, 21, 34, 55, and 89.


In [19]:
import random
import numpy as np
import torch


SEED = 3407
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


import unsloth
from datasets import load_dataset
import weave
import asyncio
from unsloth import FastLanguageModel


max_seq_length = 2048
dtype = None
load_in_4bit = False
BASE_MODEL_NAME = "unsloth/Qwen3-0.6B"
LORA_MODEL_DIR = "lora_model"
N = 30


weave.init("huggingface")


weave: Logged in as Weights & Biases user: lewis-won.
weave: View Weave data at https://wandb.ai/lewis-won-sear/huggingface/weave
INFO:weave.trace.init_message:Logged in as Weights & Biases user: lewis-won.
View Weave data at https://wandb.ai/lewis-won-sear/huggingface/weave


In [20]:
# === GLOBAL: LOAD MODELS ONLY ONCE ===
BASE_MODEL, TOKENIZER = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL_NAME,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
LORA_MODEL, _ = FastLanguageModel.from_pretrained(
    model_name=LORA_MODEL_DIR,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
BASE_MODEL.eval().to("cuda")
LORA_MODEL.eval().to("cuda")
FastLanguageModel.for_inference(BASE_MODEL)
FastLanguageModel.for_inference(LORA_MODEL)


==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.8.10: Fast Qwen3 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA GeForce RTX 4070 SUPER. Num GPUs = 1. Max memory: 11.713 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 1024, padding_idx=151654)
        (layers): ModuleList(
          (0-27): 28 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear